In [1]:
import os
import clip
import copy
import torch
from torch import nn
from datasets import build_datasets
import torch.nn.functional as F
from torchvision.models import resnet101, ResNet
from torch.utils.data import DataLoader
from clip.model import CLIP, ModifiedResNet, AttentionPool2d

b, k, c, h, w = 4, 8, 1024, 14, 14
clip_model, clip_preproess = clip.load('RN50', device=torch.device('cpu'))

In [18]:
clip_preproess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x17c31e7a0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [2]:
(dataset_train, dataset_val, dataset_test), attr_indices, class_attrs_df = build_datasets(
    dataset_dir=os.path.join('datasets', 'CUB'),
    attr_subset='cbm',
    use_class_level_attr=True,
    image_size=448
)

val_loader = DataLoader(dataset=dataset_val, batch_size=4, shuffle=True, num_workers=4)
val_loader_iter = iter(val_loader)

In [3]:
class CLIPSpatial(nn.Module):
    def __init__(self, clip_model: CLIP):
        super().__init__()
        assert type(clip_model.visual) == ModifiedResNet
        assert type(clip_model.visual.attnpool) == AttentionPool2d
        self.visual_backbone = copy.deepcopy(clip_model.visual)
        self.visual_backbone.attnpool = nn.Identity()
        self.attnpool = copy.deepcopy(clip_model.visual.attnpool)

    def forward(self, x: torch.Tensor):
        x = self.visual_backbone(x)
        b, c, h, w = x.size()  # [BS, 2048, 7, 7]
        x = x.permute(0, 2, 3, 1)  # [BS, 7, 7, 2048]
        x = x.reshape(-1, c)  # [b*7*7, 2048]
        x = x[..., None, None]  # [b*7*7, 2048, 1, 1]
        x = x.expand(-1, -1, 7, 7)  # [b*7*7, 2048, 7, 7]
        x = self.attnpool(x)  # [b*7*7, 1024]
        x = x.reshape(b, h, w, -1)  # [b, 7, 7, 1024]
        x = x.permute(0, 3, 1, 2)  # [b, 1024, 7, 7]
        return x

In [15]:
class PartCEMClip(nn.Module):
    def __init__(self, backbone: nn.Module, prototypes: torch.Tensor, *, num_parts=8, num_classes=200, dropout=0.1) -> None:
        super().__init__()
        self.k = num_parts + 1
        self.backbone = backbone
        self.dim = 1024
        self.prototypes = nn.Parameter(prototypes.unsqueeze(0)) # shape: [1, k, dim]
        self.modulations = torch.nn.Parameter(torch.ones((1, self.k, self.dim)))

        self.softmax2d = nn.Softmax2d()
        self.dropout = nn.Dropout1d(p=dropout)
        self.class_fc = nn.Linear(self.dim, num_classes)
    
    def forward(self, x):
        # Pretrained ResNet part of the model
        x = self.backbone(x)

        b, c, h, w = x.shape
        h, w = h*2, w*2
        x = torch.nn.functional.interpolate(x, size=(h, w), mode='bilinear') # shape: [b, 2048, h, w], e.g. h=w=14

        x_flat = x.view(b, c, h*w).permute(0, 2, 1) # shape: [b,h*w,c]
        x_flat_norm = F.normalize(x_flat, p=2, dim=-1) # shape: [b,h*w,c]
        proto_norm = F.normalize(self.prototypes, p=2, dim=-1) # shape: [1,k,c]
        print(x_flat_norm.isnan().any())
        print(proto_norm.isnan().any())

        maps = torch.einsum('bnc,bkc->bnk', x_flat_norm, proto_norm.expand(b, -1, -1)) # shape: [b,h*w,k]
        maps = maps.permute(0, 2, 1).reshape(b, -1, h, w) # shape: [b,k,h,w]
        maps = self.softmax2d(maps) # shape: [b,k,h,w]
        print(maps.isnan().any())

        parts = torch.einsum('bkhw,bchw->bkchw', maps, x).mean((-1,-2)) # shape: [b,k,h,w], [b,c,h,w] -> [b,k,c]
        parts_modulated = parts * self.modulations # shape: [b,k,c]
        parts_modulated_dropped = self.dropout(parts_modulated) # shape: [b,k,c]
        class_logits = self.class_fc(parts_modulated_dropped) # shape: [b,k,|y|]

        return parts, maps, class_logits

In [16]:
texts = ['back', 'beak', 'belly', 'breast', 'leg', 'tail', 'wing', 'throat', 'background']
with torch.no_grad():
    texts_tokenized = clip.tokenize(texts)
    texts_encoded = clip_model.encode_text(texts_tokenized)
clip_spatial = CLIPSpatial(clip_model)
model = PartCEMClip(backbone=clip_spatial, prototypes=texts_encoded.to(torch.float32))

In [17]:
ids, imgs, labels, attrs = next(val_loader_iter)
parts, maps, preds = model(imgs)

tensor(False)
tensor(False)
tensor(False)
